<a href="https://colab.research.google.com/github/imbishal7/aops_scraper/blob/main/aops_complete_scraper_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import numpy as np
import random
from bs4 import BeautifulSoup
import re
import time

In [ ]:
%%shell
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500

Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300

Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF
apt-get update
apt-get install chromium chromium-driver

pip install selenium

Executing: /tmp/apt-key-gpghome.Q3zbnvkX8D/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.7MtL47vwfH/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.wJf17w7bWL/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://de

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_all_elements_located as presence, all_of
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [ ]:
domain = 'https://artofproblemsolving.com'

In [ ]:
def driver_setup():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--disable-gpu")
    driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)
    
    return driver

In [ ]:
def scroll_to_end(driver, pause_time=1):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [ ]:
def give_problem_and_all_solutions(problem_url):
  
  driver = driver_setup()

  #for problem
  div_class = 'cmty-post-html'
  source_class = "cmty-topic-source-display"
  
  wait = WebDriverWait(driver, 10)
  driver.get(problem_url)

  element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"div.{div_class}")))
  soup = BeautifulSoup(driver.page_source, 'html.parser')

  links = soup.find_all('div', {'class': div_class})

  try:
    problem = links[0].prettify()
  except:
    problem = ''
  try:
    source = soup.find('div', {'class': source_class}).text
  except:
    source = ''
    
  source = source.replace('\n', '').replace('\t', '').replace('Source:','')
  info = [source, problem]


  #for solutions
  solutions = []

  soln_post_div_class = 'cmty-post'
  like_count = 'cmty-post-thank-count'
  soln = 'cmty-post-html'
  hidden_soln = 'cmty-hide-content'

  try:
    posts = soup.find_all('div', {'class': soln_post_div_class})
  except:
    posts = []

  for i in posts:

    like = i.find('span',{'class': like_count}).text
    try:
      like = int(re.findall(r'\d+', like)[0])
    except:
      like = int(0)


    solution_post = i.find('div',{'class': soln})
    try:
      hidden_post = ''
      hidden_content = solution_post.find_all('div',{'class': hidden_soln})

      for hh in hidden_content:
        hh['style'] = 'display: block;'

        hidden_post += hh.prettify()

      solution=solution_post.prettify() + hidden_post
    except:
      solution = solution_post.prettify()

    soln_with_like = [like,solution]

    if len(str(soln_with_like[1]))> 500:
      solutions.append(soln_with_like)
    else:
      pass

  only_solutions = solutions[1:]
  ranked_solutions = sorted(only_solutions, key=lambda x: x[0], reverse=True)

  no_of_solutions = len(ranked_solutions)
  if no_of_solutions>5:
    ranked_solutions = ranked_solutions[:5]
  else:
    pass
    
  filtered_solutions = [z[1] for z in ranked_solutions]

  info.extend(ranked_solutions)
  return info

In [ ]:
#Part9

In [ ]:
part = 'part_9'

In [ ]:
data = pd.read_csv(f'aops_{part}.csv')

In [ ]:
data

,Unnamed: 0.1,Unnamed: 0,category,contest,link,name,source
0,21652,21652,undergraduate_contests,Vojtěch Jarník IMC,/community/c2500213_1995_vjimc,1995 VJIMC,/community/c6h2725946p23724555
1,21653,21653,undergraduate_contests,Vojtěch Jarník IMC,/community/c2500213_1995_vjimc,1995 VJIMC,/community/c7h2725947p23724560
2,21654,21654,undergraduate_contests,Vojtěch Jarník IMC,/community/c2500213_1995_vjimc,1995 VJIMC,/community/c7h2725948p23724562
3,21655,21655,undergraduate_contests,VTRMC,/community/c694601_1979_vtrmc,1979 VTRMC,/community/c6h1687627p10778031
4,21656,21656,undergraduate_contests,VTRMC,/community/c694601_1979_vtrmc,1979 VTRMC,/community/c6h1687632p10778070
...,...,...,...,...,...,...,...
2701,24353,24353,national_olympiads,Bulgaria National Olympiad,/community/c2383849_1980_bulgaria_national_oly...,1980 Bulgaria National Olympiad,/community/c6h1504464p22332607
2702,24354,24354,national_olympiads,Bulgaria National Olympiad,/community/c2383849_1980_bulgaria_national_oly...,1980 Bulgaria National Olympiad,/community/c1068820h2593204p22357831
2703,24355,24355,national_olympiads,Bulgaria National Olympiad,/community/c2383849_1980_bulgaria_national_oly...,1980 Bulgaria National Olympiad,/community/c1068820h2593207p22357848
2704,24356,24356,national_olympiads,Bulgaria National Olympiad,/community/c2383848_1981_bulgaria_national_oly...,1981 Bulgaria National Olympiad,/community/c1068820h2593178p22357597


In [ ]:
scraped = []
skipped = []

partial = data['source']
for i in partial:

  pos = partial[partial==i].index[0]

  percent = round(pos/len(partial)*100,2)

  try:
    dig = give_problem_and_all_solutions(domain+i)
    scraped.append([i, dig])

  except:
    skipped.append(i)

  print(f'{percent} %: extracted from {pos} links. Remaining links: {len(partial)-pos}')


scraped_path = f'/content/drive/MyDrive/aops_scraped_{part}.npy'
skipped_path = f'/content/drive/MyDrive/aops_skipped_{part}.npy'

scraped_array = np.array(scraped)
skipped_array = np.array(skipped)

np.save(scraped_path, scraped_array)
np.save(skipped_path, skipped_array)

<ipython-input-5-177052b956f3>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)


0.0 %: extracted from 0 links. Remaining links: 2706
0.04 %: extracted from 1 links. Remaining links: 2705
0.07 %: extracted from 2 links. Remaining links: 2704
0.11 %: extracted from 3 links. Remaining links: 2703
0.15 %: extracted from 4 links. Remaining links: 2702
0.18 %: extracted from 5 links. Remaining links: 2701
0.22 %: extracted from 6 links. Remaining links: 2700
0.26 %: extracted from 7 links. Remaining links: 2699
0.3 %: extracted from 8 links. Remaining links: 2698
0.33 %: extracted from 9 links. Remaining links: 2697
0.37 %: extracted from 10 links. Remaining links: 2696
0.41 %: extracted from 11 links. Remaining links: 2695
0.44 %: extracted from 12 links. Remaining links: 2694
0.48 %: extracted from 13 links. Remaining links: 2693
0.52 %: extracted from 14 links. Remaining links: 2692
0.55 %: extracted from 15 links. Remaining links: 2691
0.59 %: extracted from 16 links. Remaining links: 2690
0.63 %: extracted from 17 links. Remaining links: 2689
0.67 %: extracted from

<ipython-input-13-0c3abb790688>:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  scraped_array = np.array(scraped)


In [ ]:
len(scraped_array), len(skipped_array)

(2706, 0)